## Chalenge 1
### Convert hex to base 64

I have given an hex string(base16), so i will convert it to bytes and then encode it to base64. 

In [12]:
import base64
def hexToB64(hexstr:str)->str:
    b64Bytes = base64.b64encode((bytes.fromhex(hexstr)))
    return b64Bytes.decode()

In [13]:
## Check
hexstr = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
base64str = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
print(hexToB64(hexstr)==base64str)

True


## Chalenge 2
### Fixed XOR
We have given two hex strings we have to calculate bitwise xor of those two strings

In [10]:
def xorCalc(str1:str,str2:str)->str:
    str1bytes = bytes.fromhex(str1)
    str2bytes = bytes.fromhex(str2)
    xorbytes = bytes([a^b for a,b in zip(str1bytes,str2bytes)])
    return xorbytes.hex()

In [11]:
# Check
hexstr1 = "1c0111001f010100061a024b53535009181c"
hexstr2 = "686974207468652062756c6c277320657965"
xorres = "746865206b696420646f6e277420706c6179"

print(xorCalc(hexstr1,hexstr2)==xorres)

True


## Chalenge 3
### Single-byte XOR cipher

a hex encoded string has been xored against a single character(key), i assumed that the string has been xored by repeating key to match the length of string
a single character has `8bits = 1byte` and a single hexcharacter has `4bits` so to match their length i have to multiply the character by `len(hexstr)/2`. if `len(hexstr)` is not divisible we will add a `0 bit` at starting of hexstring

In [45]:
def xor_with_character(hexstr:str,char:str)->str:
    '''This returns a string in readable format'''
    if len(hexstr)%2 != 0: hexstr = '0'+hexstr
    charbytes = bytes(char*(len(hexstr)//2),encoding='ascii')
    hexStrBytes = bytes.fromhex(hexstr)
    xored_result = bytes([a^b for a,b in zip(hexStrBytes,charbytes)])
    return xored_result.decode()

In [46]:
ciphertext = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

for i in range(ord('A'),ord('Z')+1):
    print(xor_with_character(ciphertext,chr(i))," <= result for character ",chr(i))

Zvvrpw~9TZ>j9upr|9x9ivlw}9v9{xzvw  <= result for character  A
Yuuqst}:WY=i:vsq:{:juot~:u|:x{yut  <= result for character  B
Xttpru|;VX<h;wrp~;z;ktnu;t};yzxtu  <= result for character  C
_sswur{<Q_;o<puwy<}<lsirx<sz<~}sr  <= result for character  D
^rrvtsz=P^:n=qtvx=|=mrhsy=r{=|~rs  <= result for character  E
]qquwpy>S]9m>rwu{>>nqkpz>qx>|}qp  <= result for character  F
\pptvqx?R\8l?svtz?~?opjq{?py?}~|pq  <= result for character  G
S{y~w0]S7c0|y{u0q0`e~t0v0rqs~  <= result for character  H
R~~zxv1\R6b1}xzt1p1a~du1~w1spr~  <= result for character  I
Q}}y{|u2_Q5a2~{yw2s2b}g|v2}t2psq}|  <= result for character  J
P||xz}t3^P4`3zxv3r3c|f}w3|u3qrp|}  <= result for character  K
W{{}zs4YW3g4x}q4u4d{azp4{r4vuw{z  <= result for character  L
Vzz~|{r5XV2f5y|~p5t5ez`{q5zs5wtvz{  <= result for character  M
Uyy}xq6[U1e6z}s6w6fycxr6yp6twuyx  <= result for character  N
Txx|~yp7ZT0d7{~|r7v7gxbys7xq7uvtxy  <= result for character  O
Kggcafo(EK/{(dacm(i(xg}fl(gn(jikgf  <= result for chara

so for character `X` we got our decrypted message `Cooking MC's like a pound of bacon`. now for designing a metric i will use wikipedia most used characters in english language and Bhattacharya's coefficient(inspired from `pranavmaneriker`).

In [47]:
import math

def calculate_bc(eng_str:str)->float:
    '''This function gives a value between 0 and 1, which represents similarity of eng_str with eng language'''
    char_freq = {
        'a': 0.08166999999999999,
        'b': 0.01492,
        'c': 0.02782,
        'd': 0.04253,
        'e': 0.12702,
        'f': 0.02228,
        'g': 0.02015,
        'h': 0.06094,
        'i': 0.06966,
        'j': 0.00153,
        'k': 0.00772,
        'l': 0.04025,
        'm': 0.02406,
        'n': 0.06749,
        'o': 0.07507,
        'p': 0.01929,
        'q': 0.00095,
        'r': 0.05987,
        's': 0.06326999999999999,
        't': 0.09055999999999999,
        'u': 0.02758,
        'v': 0.00978,
        'w': 0.0236,
        'x': 0.0015,
        'y': 0.01974,
        'z': 0.00074
    }
    str_chr_freq = {a:0 for a in char_freq}
    eng_str = eng_str.lower()
    for c in eng_str:
        if c in str_chr_freq:
            str_chr_freq[c] += 1
    
    BC = 0
    for i in char_freq: BC += math.sqrt(char_freq[i]*str_chr_freq[i]/len(eng_str))
    return BC

In [83]:
def sorted_xor_ciphers(ciphertext):
    poss_soln = []
    for c in range(ord('A'),ord('Z')+1):
        try:
            decoded_str = xor_with_character(ciphertext,chr(c))
            poss_soln.append((chr(c),decoded_str,calculate_bc(decoded_str)))
        except UnicodeDecodeError:
            continue
    poss_soln.sort(key=lambda x:x[2],reverse=True)
    return poss_soln
print(sorted_xor_ciphers(ciphertext)[0])

('X', "Cooking MC's like a pound of bacon", 0.7102457869430675)


## Challenge 4
### Find the XOR Ciphered string

In file `challenge4.txt` hex strings are ciphered same as above, i will decipher all the strings and will sort acc to their BC.

In [88]:
with open('challenge4.txt','r') as f:
    input_str = f.read().strip().splitlines()

output_str = []
for s in input_str:
    s = s.strip()
    output_str.extend(sorted_xor_ciphers(s))

output_str.sort(key=lambda x:x[2],reverse=True)
print(output_str[0:9])

[('B', 'WPuS[GNNS\x00f\x18B\rAt\x12\nZraH[gJ\x1eXUed', 0.6548860350981618), ('Z', 'eA\x00neY\x12hCaOf\x12uM_cuXE\x05S\tsV\x16\x19kql', 0.6445543376663577), ('P', 'HTA\x06DSO\x0f\x07O{Dl\rlKu*\x1bge_H\x14\nnXdLm', 0.6363520378308245), ('Z', 'tH\x01uvGUEMTV\x0biEV\\sLJn\x06\\Y#Ei\x7feTV', 0.6345215840255556), ('T', 'AFcEMQXXE\x16p\x0eT\x1bWb\x04\x1cLdw^Mq\\\x08NCsr', 0.627097811672793), ('E', 'PWrT\\@IIT\x07a\x1fE\nFs\x15\r]ufO\\`M\x19_Rbc', 0.6255548153740456), ('M', '{\x04lRlI\x1bHSdBV\x01\x15\x00J\x04oAeFg\x1d\x1eRkdL\x1ds', 0.6220477507384194), ('Q', 'IU@\x07ERN\x0e\x06NzEm\x0cmJt+\x1afd^I\x15\x0boYeMl', 0.6162293192216397), ('C', 'VQtRZFOOR\x01g\x19C\x0c@u\x13\x0b[s`IZfK\x1fYTde', 0.6060819316694385)]


## Challenge 5
### Implement repeating-key XOR
In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on. 

In [128]:
def repeating_key_xor(eng_str:str,key:str):
    "".join(eng_str.splitlines())
    xorbytes = []
    for i,ch in enumerate(eng_str):
        keyIndex = key[(i%len(key))]
        xorbyte = hex(ord(ch)^ord(keyIndex))[2:]
        if len(xorbyte)%2 != 0 : xorbyte = "0"+xorbyte
        xorbytes.append(xorbyte)
    res = "".join(xorbytes)
    if len(res)%2 != 0 : res = "0"+res
    return res

In [130]:
eng_txt = '''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

ciphertext = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"

print(repeating_key_xor(eng_txt,"ICE")==ciphertext)

True
